# 本地檔案分享自動化工具

這個筆記本提供 4 種不需要 Azure AD 的檔案分享方案，適用於共享磁碟機上的檔案。

## 方案概覽：
1. **PowerShell + OneDrive COM** - 使用 OneDrive 同步客戶端 API
2. **VBA 巨集** - Excel 內建自動化
3. **PowerShell 批次腳本** - 模擬分享動作
4. **Python 自動化** - 使用 pyautogui 模擬操作

## 準備工作

In [ ]:
import os
import sys
import subprocess
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# 安裝必要套件
required_packages = ['pandas', 'openpyxl', 'ipywidgets', 'pyautogui', 'pyperclip']

for package in required_packages:
    try:
        __import__(package)
        print(f"✓ {package} 已安裝")
    except ImportError:
        print(f"📦 安裝 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("\n✅ 準備完成！")

## 選項 1: PowerShell + OneDrive COM 物件

這個方法使用 OneDrive 同步客戶端的本地 API 來產生分享連結。

In [ ]:
def create_onedrive_share_script(folder_path, reviewer_emails):
    """建立使用 OneDrive COM 物件的 PowerShell 腳本"""
    
    script_content = '''# PowerShell Script - OneDrive 分享自動化
# 需要 OneDrive 已同步到本地

Add-Type -TypeDefinition @"
using System;
using System.Runtime.InteropServices;

public class OneDriveAPI {
    [DllImport("shell32.dll", CharSet = CharSet.Unicode)]
    public static extern IntPtr SHGetKnownFolderPath(
        [MarshalAs(UnmanagedType.LPStruct)] Guid rfid,
        uint dwFlags,
        IntPtr hToken,
        out IntPtr pszPath);
}
"@

# 函數：取得 OneDrive 路徑
function Get-OneDrivePath {
    $OneDriveCommercial = [Environment]::GetEnvironmentVariable("OneDriveCommercial")
    $OneDriveConsumer = [Environment]::GetEnvironmentVariable("OneDrive")
    
    if ($OneDriveCommercial) { return $OneDriveCommercial }
    elseif ($OneDriveConsumer) { return $OneDriveConsumer }
    else { return $null }
}

# 函數：建立分享連結
function Create-ShareLink {
    param(
        [string]$FilePath,
        [string]$Email
    )
    
    try {
        # 使用 Windows 10+ 的分享功能
        $shell = New-Object -ComObject Shell.Application
        $folder = $shell.Namespace((Get-Item $FilePath).DirectoryName)
        $file = $folder.ParseName((Get-Item $FilePath).Name)
        
        # 觸發分享對話框
        $file.InvokeVerb("share")
        
        Start-Sleep -Seconds 2
        
        # 嘗試使用剪貼簿取得連結
        Add-Type -AssemblyName System.Windows.Forms
        $link = [System.Windows.Forms.Clipboard]::GetText()
        
        return $link
    }
    catch {
        Write-Host "錯誤: $_" -ForegroundColor Red
        return $null
    }
}

# 主程式
$basePath = "''' + folder_path + '''"
$results = @()

Write-Host "開始處理分享設定..." -ForegroundColor Green

'''
    
    # 為每個審查者添加處理邏輯
    for reviewer, email in reviewer_emails.items():
        script_content += f'''
# 處理 {reviewer}
$reviewerPath = Join-Path $basePath "{reviewer}"
$files = Get-ChildItem -Path $reviewerPath -Filter "*.xlsx" -File

foreach ($file in $files) {{
    Write-Host "分享檔案給 {reviewer} ({email}): $($file.Name)"
    
    # 方法1: 使用檔案總管的分享功能
    $result = @{{
        Reviewer = "{reviewer}"
        Email = "{email}"
        File = $file.FullName
        Status = "Pending"
    }}
    
    # 開啟檔案位置
    Start-Process explorer.exe -ArgumentList "/select,$($file.FullName)"
    
    Write-Host "請在檔案總管中：" -ForegroundColor Yellow
    Write-Host "1. 右鍵點擊檔案" -ForegroundColor Yellow
    Write-Host "2. 選擇 '分享' 或 '共用'" -ForegroundColor Yellow
    Write-Host "3. 輸入: {email}" -ForegroundColor Yellow
    Write-Host "4. 按 Enter 繼續..." -ForegroundColor Yellow
    
    Read-Host
    
    $result.Status = "Shared"
    $results += $result
}}
'''
    
    script_content += '''
# 匯出結果
$timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
$resultFile = "share_results_$timestamp.csv"
$results | Export-Csv -Path $resultFile -NoTypeInformation

Write-Host "\n✅ 完成！結果已儲存到: $resultFile" -ForegroundColor Green
'''
    
    # 儲存腳本
    script_path = "onedrive_share.ps1"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"✅ 已建立 PowerShell 腳本: {script_path}")
    print("\n執行方式：")
    print("1. 以管理員身分開啟 PowerShell")
    print("2. 執行: Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope CurrentUser")
    print(f"3. 執行: .\\{script_path}")
    
    return script_path

## 選項 2: VBA 巨集自動化

使用 Excel VBA 來自動化分享流程。

In [ ]:
def create_vba_macro():
    """建立 VBA 巨集程式碼"""
    
    vba_code = '''Sub ShareFilesWithReviewers()
    ' VBA 巨集 - 自動分享 Excel 檔案
    
    Dim fso As Object
    Dim folder As Object
    Dim subfolder As Object
    Dim file As Object
    Dim shell As Object
    Dim reviewerEmail As String
    Dim basePath As String
    Dim logFile As String
    Dim logNum As Integer
    
    ' 設定基礎路徑
    basePath = InputBox("請輸入包含審查者資料夾的路徑:", "路徑設定")
    If basePath = "" Then Exit Sub
    
    Set fso = CreateObject("Scripting.FileSystemObject")
    Set shell = CreateObject("Shell.Application")
    
    ' 建立記錄檔
    logFile = basePath & "\\share_log_" & Format(Now, "yyyymmdd_hhmmss") & ".txt"
    logNum = FreeFile
    Open logFile For Output As #logNum
    
    Print #logNum, "分享記錄 - " & Now
    Print #logNum, "===================="
    
    ' 處理每個子資料夾
    Set folder = fso.GetFolder(basePath)
    For Each subfolder In folder.SubFolders
        ' 詢問該審查者的 email
        reviewerEmail = InputBox("請輸入 " & subfolder.Name & " 的 email:", "Email 設定")
        
        If reviewerEmail <> "" Then
            ' 處理資料夾中的每個 Excel 檔案
            For Each file In subfolder.Files
                If LCase(fso.GetExtensionName(file.Path)) = "xlsx" Then
                    ' 記錄
                    Print #logNum, "檔案: " & file.Path
                    Print #logNum, "分享給: " & reviewerEmail
                    
                    ' 方法1: 開啟檔案所在資料夾
                    shell.Open subfolder.Path
                    
                    ' 提示使用者手動分享
                    MsgBox "請在開啟的資料夾中：" & vbCrLf & _
                           "1. 右鍵點擊: " & file.Name & vbCrLf & _
                           "2. 選擇 '分享'" & vbCrLf & _
                           "3. 輸入: " & reviewerEmail & vbCrLf & _
                           "4. 完成後按確定", vbInformation, subfolder.Name
                    
                    Print #logNum, "狀態: 已處理"
                    Print #logNum, ""
                End If
            Next file
        End If
    Next subfolder
    
    Close #logNum
    
    MsgBox "分享流程完成！" & vbCrLf & "記錄檔: " & logFile, vbInformation
    
End Sub

' 輔助函數：批次讀取 email 清單
Sub LoadEmailsFromSheet()
    ' 從工作表讀取審查者和 email 對應
    Dim ws As Worksheet
    Dim lastRow As Long
    Dim i As Long
    Dim reviewerDict As Object
    
    Set reviewerDict = CreateObject("Scripting.Dictionary")
    Set ws = ActiveSheet
    
    ' 假設 A 欄是審查者，B 欄是 email
    lastRow = ws.Cells(ws.Rows.Count, "A").End(xlUp).Row
    
    For i = 2 To lastRow ' 從第 2 列開始（跳過標題）
        If ws.Cells(i, "A").Value <> "" And ws.Cells(i, "B").Value <> "" Then
            reviewerDict.Add ws.Cells(i, "A").Value, ws.Cells(i, "B").Value
        End If
    Next i
    
    ' 使用字典進行自動化分享
    Call ShareWithDictionary(reviewerDict)
End Sub

' 使用字典的分享函數
Sub ShareWithDictionary(reviewerDict As Object)
    ' 實作批次分享邏輯
    Dim key As Variant
    
    For Each key In reviewerDict.Keys
        Debug.Print "審查者: " & key & ", Email: " & reviewerDict(key)
    Next key
End Sub
'''
    
    # 儲存 VBA 程式碼
    vba_path = "share_files_macro.bas"
    with open(vba_path, 'w', encoding='utf-8') as f:
        f.write(vba_code)
    
    print(f"✅ 已建立 VBA 巨集檔案: {vba_path}")
    print("\n使用方式：")
    print("1. 開啟 Excel")
    print("2. 按 Alt+F11 開啟 VBA 編輯器")
    print("3. 插入 > 模組")
    print("4. 貼上程式碼")
    print("5. 執行 ShareFilesWithReviewers 巨集")
    
    return vba_path

## 選項 3: PowerShell 批次腳本

使用 PowerShell 和 Windows 內建功能。

In [ ]:
def create_batch_share_script(folder_path, reviewer_emails):
    """建立批次分享 PowerShell 腳本"""
    
    script_content = '''# PowerShell 批次分享腳本
# 使用 Windows 10/11 的分享功能

param(
    [string]$Mode = "Interactive"  # Interactive 或 Automated
)

# 載入必要的程序集
Add-Type -AssemblyName System.Windows.Forms
Add-Type @"
using System;
using System.Runtime.InteropServices;
using System.Text;

public class Win32 {
    [DllImport("user32.dll")]
    public static extern IntPtr FindWindow(string lpClassName, string lpWindowName);
    
    [DllImport("user32.dll")]
    public static extern bool SetForegroundWindow(IntPtr hWnd);
    
    [DllImport("user32.dll")]
    public static extern void keybd_event(byte bVk, byte bScan, uint dwFlags, UIntPtr dwExtraInfo);
}
"@

# 函數：模擬鍵盤輸入
function Send-Keys {
    param([string]$keys)
    [System.Windows.Forms.SendKeys]::SendWait($keys)
}

# 函數：開啟分享對話框
function Open-ShareDialog {
    param(
        [string]$FilePath
    )
    
    # 選擇檔案
    $shell = New-Object -ComObject Shell.Application
    $folder = $shell.Namespace((Split-Path $FilePath -Parent))
    $file = $folder.ParseName((Split-Path $FilePath -Leaf))
    
    # 開啟檔案所在資料夾並選擇檔案
    Start-Process explorer.exe -ArgumentList "/select,`"$FilePath`""
    Start-Sleep -Seconds 2
    
    # 模擬右鍵選單
    Send-Keys "+{F10}"  # Shift+F10 開啟右鍵選單
    Start-Sleep -Milliseconds 500
    
    # 尋找並選擇分享選項（可能需要調整）
    Send-Keys "h"  # 通常 'h' 是分享的快捷鍵
    Start-Sleep -Seconds 2
}

# 主要資料
$basePath = "''' + folder_path + '''"
$emailMapping = @{
'''
    
    # 添加 email 對應
    for reviewer, email in reviewer_emails.items():
        script_content += f'    "{reviewer}" = "{email}"\n'
    
    script_content += '''}

$results = @()
$timestamp = Get-Date -Format "yyyyMMdd_HHmmss"

Write-Host "===== 檔案分享批次處理 =====" -ForegroundColor Cyan
Write-Host "模式: $Mode" -ForegroundColor Yellow
Write-Host ""

# 處理每個審查者
foreach ($reviewer in $emailMapping.Keys) {
    $email = $emailMapping[$reviewer]
    $reviewerPath = Join-Path $basePath $reviewer
    
    if (Test-Path $reviewerPath) {
        Write-Host "處理審查者: $reviewer" -ForegroundColor Green
        Write-Host "Email: $email" -ForegroundColor Gray
        
        $files = Get-ChildItem -Path $reviewerPath -Filter "*.xlsx" -File
        
        foreach ($file in $files) {
            Write-Host "  檔案: $($file.Name)" -ForegroundColor White
            
            if ($Mode -eq "Interactive") {
                # 互動模式
                Open-ShareDialog -FilePath $file.FullName
                
                Write-Host "  請在分享視窗中：" -ForegroundColor Yellow
                Write-Host "    1. 輸入 email: $email" -ForegroundColor Yellow
                Write-Host "    2. 選擇權限等級" -ForegroundColor Yellow
                Write-Host "    3. 點擊分享" -ForegroundColor Yellow
                Write-Host "  完成後按 Enter 繼續..." -ForegroundColor Cyan
                Read-Host
            }
            else {
                # 自動化模式（需要更多設定）
                Write-Host "  [自動化模式尚未實作]" -ForegroundColor Red
            }
            
            # 記錄結果
            $results += [PSCustomObject]@{
                Timestamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
                Reviewer = $reviewer
                Email = $email
                File = $file.Name
                Path = $file.FullName
                Status = "Processed"
            }
        }
        Write-Host ""
    }
    else {
        Write-Host "找不到資料夾: $reviewerPath" -ForegroundColor Red
    }
}

# 儲存結果
$resultPath = Join-Path $basePath "share_results_$timestamp.csv"
$results | Export-Csv -Path $resultPath -NoTypeInformation -Encoding UTF8

Write-Host "\n===== 完成 =====" -ForegroundColor Green
Write-Host "處理了 $($results.Count) 個檔案" -ForegroundColor White
Write-Host "結果已儲存到: $resultPath" -ForegroundColor White

# 開啟結果檔案
$openResult = Read-Host "\n是否要開啟結果檔案? (Y/N)"
if ($openResult -eq "Y") {
    Start-Process $resultPath
}
'''
    
    # 儲存腳本
    script_path = "batch_share.ps1"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"✅ 已建立批次分享腳本: {script_path}")
    print("\n執行方式：")
    print(f"powershell -ExecutionPolicy Bypass -File {script_path} -Mode Interactive")
    
    return script_path

## 選項 4: Python 自動化（使用 pyautogui）

使用 Python 模擬滑鼠和鍵盤操作。

In [ ]:
import pyautogui
import pyperclip
import time
import subprocess

class ShareAutomation:
    """使用 pyautogui 自動化分享流程"""
    
    def __init__(self):
        # 設定 pyautogui
        pyautogui.FAILSAFE = True
        pyautogui.PAUSE = 0.5
        
    def open_file_location(self, file_path):
        """開啟檔案位置並選擇檔案"""
        # Windows 指令
        subprocess.Popen(f'explorer /select,"{file_path}"')
        time.sleep(3)  # 等待檔案總管開啟
        
    def right_click_and_share(self):
        """執行右鍵點擊並選擇分享"""
        # 右鍵點擊
        pyautogui.click(button='right')
        time.sleep(1)
        
        # 尋找分享選項（可能需要調整）
        # 方法1: 使用鍵盤導航
        pyautogui.press('h')  # 假設 'h' 是分享的快捷鍵
        
        # 或者方法2: 點擊分享選項（需要知道位置）
        # share_pos = pyautogui.locateOnScreen('share_button.png')
        # if share_pos:
        #     pyautogui.click(share_pos)
        
    def enter_email_and_share(self, email):
        """輸入 email 並完成分享"""
        time.sleep(2)  # 等待分享對話框
        
        # 輸入 email
        pyperclip.copy(email)  # 複製到剪貼簿
        pyautogui.hotkey('ctrl', 'v')  # 貼上
        
        time.sleep(1)
        pyautogui.press('tab')  # 移到下一個欄位
        
        # 選擇權限（如果需要）
        # pyautogui.press('down')  # 選擇編輯權限
        
        time.sleep(1)
        pyautogui.press('enter')  # 確認分享
        
    def batch_share(self, file_list, email_mapping):
        """批次分享檔案"""
        results = []
        
        print("🤖 開始自動化分享流程...")
        print("⚠️  請不要移動滑鼠！")
        print("\n按 Ctrl+C 可隨時中斷\n")
        
        # 倒數計時
        for i in range(5, 0, -1):
            print(f"開始倒數: {i}...")
            time.sleep(1)
        
        for file_info in file_list:
            reviewer = file_info['reviewer']
            file_path = file_info['path']
            email = email_mapping.get(reviewer, '')
            
            if not email:
                print(f"⚠️  跳過 {reviewer} - 沒有 email")
                continue
            
            try:
                print(f"\n處理: {reviewer} - {os.path.basename(file_path)}")
                
                # 步驟 1: 開啟檔案位置
                self.open_file_location(file_path)
                
                # 步驟 2: 右鍵分享
                self.right_click_and_share()
                
                # 步驟 3: 輸入 email
                self.enter_email_and_share(email)
                
                # 記錄成功
                results.append({
                    'reviewer': reviewer,
                    'email': email,
                    'file': file_path,
                    'status': 'Success'
                })
                
                print("✅ 完成")
                
                # 等待下一個
                time.sleep(3)
                
            except KeyboardInterrupt:
                print("\n❌ 使用者中斷")
                break
            except Exception as e:
                print(f"❌ 錯誤: {str(e)}")
                results.append({
                    'reviewer': reviewer,
                    'email': email,
                    'file': file_path,
                    'status': f'Error: {str(e)}'
                })
        
        return results

# 示範函數
def demo_python_automation():
    """示範 Python 自動化"""
    
    automation = ShareAutomation()
    
    # 測試資料
    test_files = [
        {'reviewer': '張三', 'path': 'C:\\Shared\\張三\\data.xlsx'},
        {'reviewer': '李四', 'path': 'C:\\Shared\\李四\\data.xlsx'}
    ]
    
    test_emails = {
        '張三': 'zhang.san@company.com',
        '李四': 'li.si@company.com'
    }
    
    # 執行自動化
    # results = automation.batch_share(test_files, test_emails)
    
    print("\n📌 Python 自動化設定完成")
    print("使用方式：")
    print("1. 修改 test_files 和 test_emails")
    print("2. 取消註解 results = automation.batch_share(...) 這行")
    print("3. 執行程式碼")
    print("\n注意事項：")
    print("- 確保螢幕解析度一致")
    print("- 不要在執行時移動滑鼠")
    print("- 可能需要調整時間延遲")

# 執行示範
demo_python_automation()

## 主要控制介面

統一的介面來選擇和執行不同的方案。

In [ ]:
# 建立使用者介面
folder_input = widgets.Text(
    placeholder='C:\\Shared\\ReviewerFolders',
    description='基礎資料夾:',
    layout=widgets.Layout(width='500px')
)

# Email 對應表輸入
email_text = widgets.Textarea(
    placeholder='審查者1:email1@company.com\n審查者2:email2@company.com',
    description='Email 對應:',
    layout=widgets.Layout(width='500px', height='150px')
)

# 方案選擇
method_dropdown = widgets.Dropdown(
    options=[
        ('1. PowerShell + OneDrive COM', 'onedrive'),
        ('2. VBA 巨集', 'vba'),
        ('3. PowerShell 批次腳本', 'batch'),
        ('4. Python 自動化', 'python')
    ],
    value='onedrive',
    description='選擇方案:',
    layout=widgets.Layout(width='300px')
)

# 執行按鈕
generate_button = widgets.Button(
    description='生成腳本',
    layout=widgets.Layout(width='150px')
)

output_area = widgets.Output()

def parse_email_mapping(text):
    """解析 email 對應文字"""
    mapping = {}
    for line in text.strip().split('\n'):
        if ':' in line:
            reviewer, email = line.split(':', 1)
            mapping[reviewer.strip()] = email.strip()
    return mapping

def on_generate_click(b):
    """處理生成按鈕點擊"""
    with output_area:
        clear_output()
        
        folder = folder_input.value.strip()
        if not folder:
            print("❌ 請輸入基礎資料夾路徑")
            return
        
        # 解析 email 對應
        email_mapping = parse_email_mapping(email_text.value)
        if not email_mapping:
            print("❌ 請輸入至少一組 email 對應")
            return
        
        print(f"📁 基礎資料夾: {folder}")
        print(f"📧 找到 {len(email_mapping)} 組對應")
        print("\n" + "="*50 + "\n")
        
        # 根據選擇執行
        method = method_dropdown.value
        
        if method == 'onedrive':
            create_onedrive_share_script(folder, email_mapping)
        elif method == 'vba':
            create_vba_macro()
        elif method == 'batch':
            create_batch_share_script(folder, email_mapping)
        elif method == 'python':
            print("📌 Python 自動化已在上方程式碼區塊準備")
            print("請修改 test_files 和 test_emails 變數後執行")

generate_button.on_click(on_generate_click)

# 顯示介面
display(HTML("<h3>📋 設定分享參數</h3>"))
display(folder_input)
display(email_text)
display(method_dropdown)
display(generate_button)
display(output_area)

# 顯示說明
display(HTML("""
<h3>📖 使用說明</h3>
<ol>
<li><b>基礎資料夾</b>：包含所有審查者子資料夾的路徑</li>
<li><b>Email 對應</b>：每行一組，格式為「審查者:email」</li>
<li><b>選擇方案</b>：根據您的環境選擇最適合的方案</li>
<li><b>生成腳本</b>：建立對應的腳本檔案</li>
</ol>

<h4>💡 建議：</h4>
<ul>
<li>方案 1：適合已安裝 OneDrive 同步的環境</li>
<li>方案 2：適合熟悉 Excel VBA 的使用者</li>
<li>方案 3：最通用，適合 Windows 10/11</li>
<li>方案 4：最自動化，但需要調整參數</li>
</ul>
"""))

## 疑難排解

### 常見問題

1. **找不到分享選項**
   - 確認檔案在 OneDrive/SharePoint 同步資料夾中
   - 確認 Windows 版本支援分享功能（Windows 10/11）

2. **PowerShell 執行錯誤**
   ```powershell
   Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope CurrentUser
   ```

3. **Python 自動化不準確**
   - 調整 `time.sleep()` 的延遲時間
   - 確保螢幕解析度一致
   - 使用 `pyautogui.screenshot()` 確認位置

4. **VBA 巨集被封鎖**
   - 在 Excel 信任中心啟用巨集
   - 將檔案加入信任位置

### 手動備援方案

如果所有自動化方案都失敗，可以：
1. 使用生成的 CSV 檔案作為清單
2. 手動在檔案總管中右鍵分享
3. 使用 SharePoint 網頁介面批次設定權限